In [1]:
import pandas as pd
import re
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from connexion_db import *

In [2]:
browser = webdriver.Firefox()
browser.get('https://www.tripadvisor.fr')
time.sleep(2)
browser.maximize_window()

In [3]:
try:
    cookies_btn = browser.find_element(By.ID, 'onetrust-reject-all-handler')
    cookies_btn.click()
except:
    pass

# Recherche : restaurants sur Paris

In [4]:
restaurant_btn = browser.find_element(By.CSS_SELECTOR, 'a[data-automation="centralNav_restaurants"]')
restaurant_btn.click()
time.sleep(2)

search_bar = browser.find_element(By.CSS_SELECTOR, 'input[title="Rechercher"]')
search_bar.send_keys('Paris')
time.sleep(3)

browser.find_element(By.ID, 'typeahead_results').find_element(By.TAG_NAME, 'a').click()

# Clique sur un élément

In [5]:
# Fonction pour la page avec tous les restaurants
def get_all_restaurant_divs(browser):
    """Prend en paramètre un WebDriver et renvoie une liste de tous les restaurants (div en HTML).

    Exemple d'utilisation : `get_all_restaurant_divs(browser)`
    """
    elements = browser.find_element(By.CSS_SELECTOR, 'div[class="Ikpld f e"]').find_elements(By.CSS_SELECTOR, 'div[data-test]')
    return [el for el in elements if re.search(r'\d+_list_item', el.get_attribute('data-test'))]

In [6]:
# Fonctions pour la page d'un restaurant

# TODO: modifier pour utiliser une classe ?
def get_restaurant_page_info(browser):
    """Prend en paramètre un WebDriver et récupère le nom, les catégories et l'adresse sur la page d'un restaurant et les renvoie dans un dictionnaire.

    Exemple d'utilisation : `get_restaurant_page_info(browser)`

    Exemple de résultat :
    ```json
    {
      'nom': 'Le Jules Verne',
      'categorie': ['Française', 'Européenne'],
      'adresse': 'Avenue Gustave Eiffel, 75007 Paris France',
      'avis': []
    }
    ```
    """
    res_info_div = browser.find_element(By.CSS_SELECTOR, 'div[data-test-target="restaurant-detail-info"]')

    categories = res_info_div.find_elements(By.CLASS_NAME, 'dlMOJ')

    return {
        'nom': res_info_div.find_element(By.TAG_NAME, 'h1').text,
        'categorie': [cat.text for cat in categories if '€' not in cat.text],
        'adresse': res_info_div.find_elements(By.CLASS_NAME, 'AYHFM')[1].text,
        'avis': []
    }

def get_all_review_divs(browser):
    """Prend en paramètre un WebDriver et renvoie une liste de tous les avis (div en HTML).
    
    Exemple d'utilisation : `get_all_review_divs(browser)`
    """
    return browser.find_element(By.CSS_SELECTOR, 'div[data-contextchoice="DETAIL"]').find_elements(By.CSS_SELECTOR, 'div[class="reviewSelector"]')

def get_review_info(review_div):
    """Prend en paramètre un avis (div en HTML) et renvoie un dictionnaire avec sa note, son titre et son texte.
    
    Exemple d'utilisation : 
    ```python
    liste_avis = browser.find_elements(By.CSS_SELECTOR, 'div[class="reviewSelector"]')
    avis = get_avis_info(liste_avis[0])
    ```
    
    Exemple de résultat :
    ```json    
    {
      'note': '50',   
      'titre': 'Très bon restaurant',
      'texte': 'Les plats étaient très bons et le service était excellent.'
    }
    ```
    """
    classe_note = review_div.find_element(By.CLASS_NAME, 'ui_bubble_rating').get_attribute('class').split('_')[-1]

    return {
        'note': int(classe_note) / 10,
        'titre': review_div.find_element(By.CLASS_NAME, 'noQuotes').text,
        'texte': review_div.find_element(By.CLASS_NAME, 'partial_entry').text
    }

In [7]:
def switch_to_next_restaurants_page(browser):
    """Prend en paramètre un WebDriver et clique sur le bouton \"Suivant\" s'il existe.

    Exemple d'utilisation : `switch_to_next_restaurants_page(browser)`
    """
    try:
        next_page_btn = browser.find_element(By.CSS_SELECTOR, 'a[data-smoke-attr="pagination-next-arrow"]')
        
        if next_page_btn:
            browser.execute_script(f'window.scrollTo(0, {next_page_btn.rect["y"] - 300})')

            next_page_btn.click()
    except Exception as e:
        print('Erreur lors de la tentative de changement de page :', e)
        pass

In [8]:
def recup_avis(restaurant_id, browser):
    """Prend en paramètre l'ID du restaurant et un WebDriver.
    Permet de récupérer les avis des restaurants dans la page d'un restaurant.
    
    Exemple d'utilisation : `liste_avis = recup_avis(res_id, browser)`
    
    Exemple de résultat :
    ```json
    [
      {
        'note': 5.0,   
        'titre': 'Très bon restaurant',
        'texte': 'Les plats étaient très bons et le service était excellent.'
      }
    ]
    ```"""
    page_avis = 1
    liste_avis = []

    # while True:
    while page_avis <= 5:
        # Cliquer sur tous les "Plus" pour afficher les avis en entier
        try:
            voir_plus = browser.find_elements(By.CSS_SELECTOR, 'span.taLnk.ulBlueLinks')
            for voir in voir_plus:
                voir.click()
                time.sleep(2)
        except:
            pass

        # Rafraîchir la liste des avis après chaque changement de page
        avis_html = get_all_review_divs(browser)

        for avis in avis_html:
            browser.execute_script(f'window.scrollTo(0, {avis.rect["y"] - 300})')
            time.sleep(1)

            # Récupération des informations de l'avis
            avis_dict = get_review_info(avis)
            # Ajout de l'avis à la liste
            liste_avis.append(avis_dict) 
        
        # Ajout des avis en base
        add_reviews_to_restaurant(restaurant_id, liste_avis)

        # Changement de page
        try:
            pagination = browser.find_element(By.CLASS_NAME, 'ui_pagination')
            next_btn = pagination.find_element(By.CLASS_NAME, 'next')

            if next_btn and 'disabled' not in next_btn.get_attribute('class'):
                browser.execute_script(f'window.scrollTo(0, {next_btn.rect["y"] - 300})')

                next_btn.click()

                page_avis += 1
                liste_avis = []
                time.sleep(3)
            else:
                break
        except Exception as e:
            print('Erreur lors de la tentative de changement de page :', e)
            break

    return avis_dict

In [9]:
index_restaurants = 0
page_restaurants = 1

avis = []

while True:

    # Rafraîchir la liste des restaurants après chaque changement de page
    restaurants_html = get_all_restaurant_divs(browser)

    while index_restaurants < len(restaurants_html):
        try:
            res = restaurants_html[index_restaurants]
            lien_avis = res.find_element(By.TAG_NAME, 'a')

            browser.execute_script(f'window.scrollTo(0, {lien_avis.rect["y"] - 300})')
            time.sleep(2)

            browser.get(lien_avis.get_attribute('href'))
            time.sleep(4)

            # Récupération des informations du restaurant
            res_dict = get_restaurant_page_info(browser)
            try:
                # Ajout du restaurant en base
                print(f'Ajout de {res_dict["nom"]} en base')
                res_id = add_restaurant(res_dict)

                # Récupération des avis
                avis_html = get_all_review_divs(browser)

                recup_avis(res_id, browser)
            
            except Exception as e:
                print(e)
                pass


            # Retour à la liste des restaurants
            browser.back()
            time.sleep(2)

            index_restaurants += 1

        # Si la liste de restaurants (div) est obsolète, rafraîchir
        except StaleElementReferenceException:
            restaurants_html = get_all_restaurant_divs(browser)
            continue

    # Si on arrive au bout de la liste des restaurants, on passe à la page suivante
    if index_restaurants >= len(restaurants_html):
        switch_to_next_restaurants_page(browser)
        page_restaurants += 1
        time.sleep(4)
        index_restaurants = 0


Ajout de Bao & ME en base
Ajout de Orgueil en base
Ajout de Pardi en base
Ajout de Chez Arnaud en base
Ajout de Midi 12 en base


KeyboardInterrupt: 